# Welcome to 'Security system for the blind'

Currently, it is difficult for blind people to tell who is at their door without the help of an external system. They are capable of hearing but with voice recording and maniupulation there is a window of opportunity for people to take advantage of them. We will be using face recognition with a security camera to detect people and try to identify their faces. We will then be using the 'pyttsx3' to convert the name of the person detected to speech for the blind person. 

In order to detect the faces from the security camera we will be using the 'face-detection-adas', 'face-reidentification-retail-0095', 'landmarks-regression-retail-0009' pre-trained models from the OpenVINO model zoo. You can find out more about it here ->https://docs.openvinotoolkit.org/2019_R1/usergroup1.html


In [1]:
import cv2
import numpy as np
from utils.opv import OpvModel
from sklearn.metrics.pairwise import cosine_similarity

## Task: Load the pretrained model 

In [3]:
mymodel0 = OpvModel("face-detection-adas-0001", device="CPU", fp="FP16")
mymodel1 = OpvModel("landmarks-regression-retail-0009", device="CPU", fp="FP32")
mymodel2 = OpvModel("face-reidentification-retail-0095", device="CPU", fp="FP32")

## Task: The main code

Here is the main code. It turns on our webcam, capture an image upon user request, process that image by calling other user-defined functions, and compare the resulting vector into our vector database. 

This code should be activated the last, after running the tabs below it. 

In [ ]:
cam = cv2.VideoCapture(0)
cv2.namedWindow("Security Camera")
img_counter = 0

while True:
    ret, frame = cam.read()
#     global frame
       
    cv2.putText(frame, 'Press Spacebar!', (50, 100), # include text
        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255,255), 2)  
    cv2.imshow("Security Camera", frame)   
    
    if not ret:
        break
    k = cv2.waitKey(1)
    
    if k%256 == 27: # ESC pressed       
        print("Escape hit, closing...")
        break
    elif k%256 == 32:

        
        img_name = "image_{}".format(img_counter)
        #cv2.imwrite(img_name, frame)
        print("{} captured!".format(img_name))
        try:
            predictions = mymodel0.Predict(frame)        
            DrawBoundingBoxes(predictions,frame)
            WarpPhoto(face_box) 
            GetVec(dst)
            cosim(prediction1_flat)
        except:
            print("There is no recognized face detected")
        img_counter += 1
        
cam.release()

cv2.destroyAllWindows()

## Task: Draw Bounding Box around the Face

We create a function to draw a bounding box around the face image that has been detected by face-detection-adas-0001.

Additionally, we will store the captured image of a face into the variable face_box for further processing.

In [3]:
def DrawBoundingBoxes(predictions, image, conf=.2):
    canvas = image.copy()                             # copy instead of modifying the original image
    predictions_1 = predictions[0][0]                 # subset dataframe
    confidence = predictions_1[:,2]                   # getting conf value [image_id, label, conf, x_min, y_min, x_max, y_max]
    topresults = predictions_1[(confidence>conf)]     # choosing only predictions with conf value bigger than treshold
    (h,w) = canvas.shape[:2]                        # 
    for detection in topresults:
        
        box = detection[3:7] * np.array([w, h, w, h]) # determine box location
        (xmin, ymin, xmax, ymax) = box.astype("int") # assign box location value to xmin, ymin, xmax, ymax

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4)  # make a rectangle
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin), # include text
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
        cv2.putText(canvas, str(len(topresults))+" face(s) detected", (50,50), # include text
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    
    global face_box
    face_box = canvas[ymin:ymax,xmin:xmax]
    
    cv2.imshow("face_box",face_box)
    cv2.waitKey(10)         
    #Show result image for 0,5 sec on screen
    return 

## Task: Create a Function to Warp Photo Given the Reference Points

Here, we create a function that does several things:
- find out the 5 landmarks within the face image: left and right eyes, nose, left and right corner of lips
- using perspective transformation to warp the photo into the format required by the face reidentification model
- store the new image in the variable called dst

In [4]:
def WarpPhoto(src):
    predictions = mymodel1.Predict(src)
    
    DIM = src.shape[0] 
    
    l_eye = tuple([int(DIM*i) for i in (predictions[0,0,0,0], predictions[0,1,0,0])])   
    r_eye = tuple([int(DIM*i) for i in (predictions[0,2,0,0], predictions[0,3,0,0])]) 
    nose = tuple([int(DIM*i) for i in (predictions[0,4,0,0], predictions[0,5,0,0])]) 
    l_lip = tuple([int(DIM*i) for i in (predictions[0,6,0,0], predictions[0,7,0,0])]) 
    r_lip = tuple([int(DIM*i) for i in (predictions[0,8,0,0], predictions[0,9,0,0])]) 
    
    pts1 = np.float32([l_eye,r_eye,l_lip,r_lip])
    pts2 = np.float32([[DIM*0.31556875000000000, DIM*0.4615741071428571],\
                   [DIM*0.68262291666666670, DIM*0.4615741071428571],\
                   [DIM*0.34947187500000004, DIM*0.8246919642857142],\
                   [DIM*0.65343645833333330, DIM*0.8246919642857142]])

    M = cv2.getPerspectiveTransform(pts1,pts2)
    
    global dst
    dst = cv2.warpPerspective(src,M, (DIM,DIM))
    
    cv2.imshow('Warp', dst)
    cv2.waitKey(10)
    return   

## Task: Get Face Vector from Warped Photos

Once we have the image processed to the format we want, we will now use the face-reidentification model to create a vector corresponding to the processed image. The idea is that every face will have different vectors. 

In [5]:
def GetVec(img):
    prediction1 = mymodel2.Predict(img)
    
    global prediction1_flat
    prediction1_flat = prediction1.flatten()
    prediction1_flat = prediction1_flat.reshape(1, 256)
    
    print (prediction1_flat)
    
    return prediction1_flat

## Task: Import Vector Database

Here, we import our vector database that contains the vector set of person's photos

In [6]:
import pickle
with open('./resources/database/arihant.pickle', 'rb') as f:
    predictiona1_flat = pickle.load(f)
    
with open('./resources/database/daniel.pickle', 'rb') as f:
    predictiond1_flat = pickle.load(f)
    
with open('./resources/database/arvin.pickle', 'rb') as f:
    predictionav1_flat = pickle.load(f)

## Task: Compare Vector with Database and output the name in a sound format.

In [7]:
import pyttsx3
engine = pyttsx3.init()


def cosim(vec):
    sim = cosine_similarity(vec, predictiond1_flat)
    
    if cosine_similarity(vec, predictiond1_flat) > 0.5: #if cosine similarity with daniel is high
        friends = cv2.imread("./resources/images/Daniel/Daniel.jpeg")
        cv2.imshow('It is Daniel', friends)
        engine.say("It is Daniel")  # using the engine here
        engine.runAndWait()
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    elif cosine_similarity(vec, predictiona1_flat) > 0.5: #if cosine similarity with Arihant is high
        friends = cv2.imread("./resources/images/Arihant/Arihant.jpeg")
        cv2.imshow('It is Arihant', friends)
        engine.say("It is Arihant")
        engine.runAndWait()
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    elif cosine_similarity(vec, predictionav1_flat) > 0.5: #if cosine similarity with Arihant is high
        friends = cv2.imread("./resources/images/Arvin/Arvin.jpg")
        cv2.imshow('It is Arvin', friends)
        engine.say("It is Arvin")
        engine.runAndWait()
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else : #if cosine similarity with daniel is high
        print("Do you belong here?")
        friends = cv2.imread("./resources/images/denied.jpeg")
        cv2.imshow('Do you belong here?', friends)
        engine.say("Not found")   # using the engine here
        engine.runAndWait() 
        cv2.waitKey(0)
        cv2.destroyAllWindows()